In [22]:
import pandas as pd

In [23]:
datapetshop = pd.read_csv('1655023Pet shop.csv')
datapetshop.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size
0,1,67648,health,off_brand,3.5,12,33.0,42.3,852
1,2,120421,other,off_brand,4.5,10,16.0,54.3,1045
2,3,95242,toys,off_brand,2.9,6,25.0,88.2,1461
3,4,105632,health,off_brand,2.6,8,NaN,5.0,1579
4,5,221627,toys,off_brand,4.9,11,25.0,57.0,2643


In [24]:
#assignment 1 (15%)
#

In [25]:
pip install statsmodels

704.97s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [26]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [27]:
product_category_dummies = pd.get_dummies(datapetshop['product_category'])
product_quality_dummies = pd.get_dummies(datapetshop['quality'])
datapetshop_dummies = pd.concat([datapetshop,product_category_dummies,product_quality_dummies],axis=1)
datapetshop_dummies.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium
0,1,67648,health,off_brand,3.5,12,33.0,42.3,852,False,True,False,False,True,False
1,2,120421,other,off_brand,4.5,10,16.0,54.3,1045,False,False,True,False,True,False
2,3,95242,toys,off_brand,2.9,6,25.0,88.2,1461,False,False,False,True,True,False
3,4,105632,health,off_brand,2.6,8,NaN,5.0,1579,False,True,False,False,True,False
4,5,221627,toys,off_brand,4.9,11,25.0,57.0,2643,False,False,False,True,True,False


In [28]:
# Calculate the number of categories in 'product_category'
category_counts = datapetshop['product_category'].value_counts()
 
category_counts

product_category
health    858
toys      824
other     296
food      272
Name: count, dtype: int64

In [29]:
# I'm using Health as my reference category because it has the most entries in the dataset. My first assumption will be that it sells the best aswell and that makes the regression model
# the most interpretable with the numbers
model1 = smf.ols('products_sold ~ health + food + other + toys + satisfaction + discount + retail_price + perc_physical + market_size + off_brand + premium', data = data_petshop_dummies).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:          products_sold   R-squared:                       0.783
Model:                            OLS   Adj. R-squared:                  0.782
Method:                 Least Squares   F-statistic:                     730.0
Date:                Tue, 02 Apr 2024   Prob (F-statistic):               0.00
Time:                        11:10:43   Log-Likelihood:                -23415.
No. Observations:                2037   AIC:                         4.685e+04
Df Residuals:                    2026   BIC:                         4.691e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -8.951e+04   4294.60

In [30]:
# Check for missing values in the dataset
missing_values = datapetshop_dummies.isnull().sum()
missing_values

Unnamed: 0            0
products_sold         0
product_category      0
quality             111
satisfaction          0
discount              0
retail_price        157
perc_physical        64
market_size           0
food                  0
health                0
other                 0
toys                  0
off_brand             0
premium               0
dtype: int64

In [36]:
# Dropping the already existing missing values from the dataset. Afterwards we perform CooksD.
datapetshop_dummies_nomissingvalues = datapetshop_dummies.dropna()

In [37]:
# Before we do CooksD, we will generate another regression model to perform it on without the missing values of the original dataset
model2 = smf.ols('products_sold ~ health + food + other + toys + satisfaction + discount + retail_price + perc_physical + market_size + off_brand + premium', data = datapetshop_dummies_nomissingvalues).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:          products_sold   R-squared:                       0.783
Model:                            OLS   Adj. R-squared:                  0.782
Method:                 Least Squares   F-statistic:                     774.5
Date:                Tue, 02 Apr 2024   Prob (F-statistic):               0.00
Time:                        11:12:06   Log-Likelihood:                -22325.
No. Observations:                1942   AIC:                         4.467e+04
Df Residuals:                    1932   BIC:                         4.473e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -6.183e+04   2811.44

In [38]:
# CooksD
CooksD = model2.get_influence().cooks_distance
n = len(datapetshop_dummies_nomissingvalues)
datapetshop_dummies_nomissingvalues['outlier'] = CooksD[0] > 4/n
datapetshop_dummies_nomissingvalues.head()

/tmp/ipykernel_2013/1453356653.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datapetshop_dummies_nomissingvalues['outlier'] = CooksD[0] > 4/n


,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium,outlier
0,1,67648,health,off_brand,3.5,12,33.0,42.3,852,False,True,False,False,True,False,False
1,2,120421,other,off_brand,4.5,10,16.0,54.3,1045,False,False,True,False,True,False,False
2,3,95242,toys,off_brand,2.9,6,25.0,88.2,1461,False,False,False,True,True,False,False
4,5,221627,toys,off_brand,4.9,11,25.0,57.0,2643,False,False,False,True,True,False,False
5,6,55492,other,off_brand,3.6,8,19.0,40.8,915,False,False,True,False,True,False,False


In [39]:

datapetshop_dummies_nomissingvalues_true = datapetshop_dummies_nomissingvalues[datapetshop_dummies_nomissingvalues.outlier == True]
datapetshop_dummies_nomissingvalues_false = datapetshop_dummies_nomissingvalues[datapetshop_dummies_nomissingvalues.outlier == False]

In [40]:
datapetshop_dummies_nomissingvalues_true.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium,outlier
25,26,146040,other,off_brand,4.8,224,11.0,41.9,873,False,False,True,False,True,False,True
93,94,38877,food,premium,3.2,193,17.0,67.5,654,True,False,False,False,False,True,True
104,105,103302,other,premium,2.4,5,34.0,50.7,2032,False,False,True,False,False,True,True
147,148,165368,health,off_brand,3.7,2,17.0,45.4,4179,False,True,False,False,True,False,True
228,229,98069,toys,off_brand,2.0,8,25.0,90.7,1085,False,False,False,True,True,False,True


In [41]:
print(datapetshop_dummies_nomissingvalues[datapetshop_dummies_nomissingvalues.outlier == True])

      Unnamed: 0  products_sold product_category    quality  satisfaction  \
25            26         146040            other  off_brand           4.8   
93            94          38877             food    premium           3.2   
104          105         103302            other    premium           2.4   
147          148         165368           health  off_brand           3.7   
228          229          98069             toys  off_brand           2.0   
248          249         191737             toys  off_brand           4.9   
267          268         209891           health  off_brand           4.7   
296          297         224862             toys    premium           4.8   
306          307         135531             toys  off_brand           4.2   
330          331          84358           health    premium           2.3   
441          442          36916            other    premium           4.2   
487          488         127790             food  off_brand           4.6   

In [42]:
# I analyzed the outliers. There were 60 of them, most of them could not be labeled as impossible. But there were some rows in which the discount percentage or
# the perc_physical percentage exceeded 100.
# This is impossible, thus, we will turn every value of those columns above 100 to a missing value before progressing with the assignment.
# Furthermore, the assignment tells us explicitly to return to the original dataset. So we will be going back at it again with datapetshop_dummies.

In [43]:
datapetshop_dummies.loc[datapetshop_dummies['discount']>100,'discount'] = np.nan
datapetshop_dummies.loc[datapetshop_dummies['perc_physical']>100,'perc_physical'] = np.nan

In [45]:
# Extra missing values have been added
datapetshop_dummies.isnull().sum()

Unnamed: 0            0
products_sold         0
product_category      0
quality             111
satisfaction          0
discount             22
retail_price        157
perc_physical        89
market_size           0
food                  0
health                0
other                 0
toys                  0
off_brand             0
premium               0
dtype: int64

In [ ]:
# To deal with the categorical missing data, we add a dummy missing variable to use in regression models.

In [48]:
ds2 = pd.get_dummies(datapetshop_dummies, dummy_na=True)

In [ ]:
# There were no missing values for product_category. Just to make the dataset as clean as possible we will remove some of the columns which were created through the code above.
# We only need to keep quality_nan, the rest is redundant.

In [49]:
columns_to_drop = ['product_category_food', 'product_category_health', 'product_category_other', 'product_category_toys', 'quality_off_brand', 'quality_premium', 'product_category_nan']
ds2 = ds2.drop(columns=columns_to_drop)
 
ds2.head()

,Unnamed: 0,products_sold,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium,quality_nan
0,1,67648,3.5,12.0,33.0,42.3,852,False,True,False,False,True,False,False
1,2,120421,4.5,10.0,16.0,54.3,1045,False,False,True,False,True,False,False
2,3,95242,2.9,6.0,25.0,88.2,1461,False,False,False,True,True,False,False
3,4,105632,2.6,8.0,NaN,5.0,1579,False,True,False,False,True,False,False
4,5,221627,4.9,11.0,25.0,57.0,2643,False,False,False,True,True,False,False


In [50]:
# For the continouos missing data we impute the data using IterativeImputer
 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
 
imputed_data = IterativeImputer().fit_transform(ds2)
imputed_data = pd.DataFrame(imputed_data, columns=ds2.columns)

In [52]:
# We will now check for multicollinearity using scipy.stats
 
import scipy.stats
 
import pandas as pd
from scipy.stats import pearsonr

# Select only the numerical columns (excluding dummy variables)
numerical_columns = imputed_data.select_dtypes(include=['int64', 'float64']).columns
 
# Initialize a DataFrame to hold the Pearson correlation results
pearson_correlations = pd.DataFrame(index=numerical_columns, columns=numerical_columns)
 
# Calculate Pearson correlation coefficient for each pair of numerical variables
for col1 in numerical_columns:
    for col2 in numerical_columns:
        if col1 != col2:
            # Calculate Pearson's r
            r, _ = pearsonr(imputed_data[col1], imputed_data[col2])
            pearson_correlations.at[col1, col2] = r
        else:
            # A variable's correlation with itself is always 1
            pearson_correlations.at[col1, col2] = 1.0
 
# Output the correlation matrix
print(pearson_correlations)

              Unnamed: 0 products_sold satisfaction  discount retail_price  \
Unnamed: 0           1.0       0.03433      0.03328    0.0136     0.014372   
products_sold    0.03433           1.0     0.476438  0.186052    -0.060727   
satisfaction     0.03328      0.476438          1.0   0.01558    -0.014456   
discount          0.0136      0.186052      0.01558       1.0      0.00798   
retail_price    0.014372     -0.060727    -0.014456   0.00798          1.0   
perc_physical  -0.006938      0.028243     0.029487 -0.004316     0.020455   
market_size     0.019612      0.708472    -0.003321 -0.032065       0.0322   
food            0.022291      0.137413     0.011887 -0.000179     0.021595   
health         -0.001731      0.042056      0.03206  0.021224     0.005771   
other           0.003644     -0.032073     -0.03912 -0.000204    -0.034398   
toys           -0.015895     -0.112881     -0.01292 -0.021133     0.003702   
off_brand      -0.009444      0.103354     0.029398 -0.009553   

In [54]:
# Checking individual PearsonR results to check the validity of the results from the model above.
SD = scipy.stats.pearsonr(imputed_data['satisfaction'],imputed_data['discount'])
RP_MS = scipy.stats.pearsonr(imputed_data['retail_price'],imputed_data['market_size'])
PP_RP = scipy.stats.pearsonr(imputed_data['perc_physical'],imputed_data['retail_price'])
print(SD)
print(RP_MS)
print(PP_RP)

PearsonRResult(statistic=0.015579594730495368, pvalue=0.4601260969881673)
PearsonRResult(statistic=0.03220026203044218, pvalue=0.12677600406068995)
PearsonRResult(statistic=0.020454614142629123, pvalue=0.3321412910378465)
